In [1]:
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import SubprocVecEnv
from gymnasium.envs.registration import register

from gymnasium import spaces
import torch as th
from torch import nn
import numpy as np
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

import gymnasium as gym
import os
from stable_baselines3.common.evaluation import evaluate_policy
from gym_data.envs import CarcassoneEnv

register(id='Carcassone-v0',entry_point='gym_data.envs:CarcassoneEnv',) 

In [2]:
def linear_schedule(initial_value):
    """
    Linear learning rate schedule.
    :param initial_value: (float or str)
    :return: (function)
    """
    if isinstance(initial_value, str):
        initial_value = float(initial_value)

    def func(progress):
        """
        Progress will decrease from 1 (beginning) to 0
        :param progress: (float)
        :return: (float)
        """
        return progress * initial_value

    return func

In [3]:
iter = 0

env = make_vec_env(CarcassoneEnv, seed=1,  n_envs=16, vec_env_cls=SubprocVecEnv)

model = A2C.load("pretrained_carcassone3", env, verbose=1,tensorboard_log="runs/", device="auto", learning_rate=linear_schedule(0.0001))
import time

start = time.time()
model.learn(total_timesteps=10_000_000)

end = time.time()

elapsed_time = end - start

print("Elapsed time: ", elapsed_time // 60, " minutes")

model.save("model_carcassone")
iter +=1

Total concat size:  1664
Logging to runs/A2C_15
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 101      |
|    ep_rew_mean        | -95      |
| time/                 |          |
|    fps                | 273      |
|    iterations         | 100      |
|    time_elapsed       | 5987     |
|    total_timesteps    | 1638400  |
| train/                |          |
|    entropy_loss       | -9.42    |
|    explained_variance | 0.000805 |
|    learning_rate      | 8.38e-05 |
|    n_updates          | 99       |
|    policy_loss        | -0.227   |
|    value_loss         | 157      |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 101      |
|    ep_rew_mean        | -94      |
| time/                 |          |
|    fps                | 315      |
|    iterations         | 200      |
|    time_elapsed       | 10390    |
|    total_timesteps    | 3

In [4]:
model.save("model_carcassone")

In [7]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100)

print(f"Mean Reward: {mean_reward}")

Mean Reward: -93.0
